## Customer-Intention-Prediction-ANN

Predict whether a customer is willing to extend his or her subscription at the end of month based on bank balance from a csv file.

### Import libraries

In [70]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

### Read Dataset

In [71]:
dataset = pd.read_csv("customer-data.csv")
print("Original Data:")
dataset.head()

Original Data:


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [72]:
X = dataset.drop(labels=['CustomerId', 'Surname', 'RowNumber', 'Exited'], axis=1)
Y = dataset['Exited']

### Preprocess Data

Drop the fields that are not needed for training and one-hot encode categorical data(Geography).

In [73]:
genderLabel = LabelEncoder()
X['Gender'] = genderLabel.fit_transform(X["Gender"])
X = pd.get_dummies(X, drop_first=True, columns=['Geography'])
print("Processed data:")
X.head()

Processed data:


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,1


In [74]:
Y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

Split training and test datasets.

In [75]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0, stratify=Y)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)

X_train shape: (8000, 11)
X_test shape: (2000, 11)
Y_train shape: (8000,)
Y_test shape: (2000,)


Standardize data.

In [76]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Model Building and Compilation

In [77]:
model = Sequential()
model.add(Dense(X.shape[1], activation='relu', input_dim=X.shape[1]))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [78]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 11)                132       
_________________________________________________________________
dense_19 (Dense)             (None, 128)               1536      
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 129       
Total params: 1,797
Trainable params: 1,797
Non-trainable params: 0
_________________________________________________________________


In [79]:
model.fit(X_train, Y_train.to_numpy(), batch_size=10, epochs=15, verbose=1)

Train on 8000 samples
Epoch 1/15
8000/8000 [==============================] - 3s 390us/sample - loss: 0.4618 - accuracy: 0.8048
Epoch 2/15
8000/8000 [==============================] - 2s 237us/sample - loss: 0.3807 - accuracy: 0.8409
Epoch 3/15
8000/8000 [==============================] - 2s 240us/sample - loss: 0.3524 - accuracy: 0.8539
Epoch 4/15
8000/8000 [==============================] - 2s 237us/sample - loss: 0.3437 - accuracy: 0.8580
Epoch 5/15
8000/8000 [==============================] - 2s 237us/sample - loss: 0.3391 - accuracy: 0.8612
Epoch 6/15
8000/8000 [==============================] - 2s 234us/sample - loss: 0.3355 - accuracy: 0.8609
Epoch 7/15
8000/8000 [==============================] - 2s 236us/sample - loss: 0.3335 - accuracy: 0.8594
Epoch 8/15
8000/8000 [==============================] - 2s 236us/sample - loss: 0.3308 - accuracy: 0.8631
Epoch 9/15
8000/8000 [==============================] - 2s 238us/sample - loss: 0.3308 - accuracy: 0.8634
Epoch 10/15
8000/8000 [=

In [80]:
Y_pred = model.predict_classes(X_test)
model.evaluate(X_test, Y_test.to_numpy())

2000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.34813072967529296, 0.8575]

In [81]:
confusion_matrix(Y_test, Y_pred)

array([[1538,   55],
       [ 230,  177]], dtype=int64)